In [1]:
import plotly.graph_objects as go
import plotly.express as px

In [2]:
def Leslie(fecundity, survival):
    n = len(fecundity)
    survival = survival[:n-1]
    L = zero_matrix(QQ, n)
    for i, b in enumerate(fecundity):
        L[0, i] = b
    for i, s in enumerate(survival):
        L[i + 1, i] = s
    return(L)



def is_essentially_real(x):
    if x.imag() == 0:
        return(True)
    else:
        return(False)
    
#Use of "if is_essentiallY_real(e)" rather than "if e in RR" is required since some computational errors
#    seem to come up in the eigenvalue computation, giving us things like x + 0.?e-80*I.

def get_leading_eigenvalue(L):
    evals = L.eigenvalues()
    moduli = [e.n() for e in evals if is_essentially_real(e)]
    moduli = [e for e in moduli if e >= 0]
    r = max(moduli)
    return(r)

def normalize(vec):
    tot = sum(vec)
    vec = vec/tot
    return(vec)

def get_leading_evec(L):
    r = get_leading_eigenvalue(L)
    evecs = L.eigenvectors_right()
    arrs = [e[0].n() for e in evecs]
    i = arrs.index(r)
    leading_evec = evecs[i][1][0]
    leading_evec = normalize(leading_evec).n()
    return(leading_evec)


In [3]:
def hyperbolic_survival(s0, a):
    s = (a - (a - 1)*s0)/(a+1 - (a)*s0)
    return(s)

In [4]:
def increased_fec(fec, by, at):
    new_fec = deepcopy(fec)
    new_fec[at] = fec[at] + by
    return(new_fec)

def increased_fitness(inc_fec, surv):
    L_inc = Leslie(inc_fec, surv)
    r = get_leading_eigenvalue(L_inc)
    return r


In [5]:
def required_increase(fec, surv, at, r0):
    '''
    For given fecundity and survival, returns required increase to fecundity at "at" to increase fitness to r0
    '''
    def g(b):
        inc_fec = increased_fec(fec, b, at)
        r1 = increased_fitness(inc_fec, surv)
        return r1 - r0
#    F = bisection(g, 0, 20)   #Required before (see sagemath ask.com for my question and how it was fixed)
    F = find_root(g, 0, 200000 )
    return(F)

In [6]:
def growth_of_P(fec, surv, P, verbose = False):
    n = len(fec)
    appreciation_vector = [0]*n
    appreciation_vector[0] = P
    inc_fec0 = increased_fec(fec, P, 0)
    r0 = increased_fitness(inc_fec0, surv)
    for a in range(1, floor(n)):
        F_a = required_increase(fec, surv, a, r0)
        appreciation_vector[a] = F_a
        if verbose:
            print('appreciated to {}'.format(a))
    return(appreciation_vector)

def plot_growth_of_P(fec, surv, P, verbose = False):
    app_vec = growth_of_P(fec, surv, P, verbose)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(len(app_vec))), y=app_vec,
                    mode='markers'))
    fig.show()

In [20]:
surv = [hyperbolic_survival(1/2, a) for a in range(10)]

def get_fec_vector(r_desired, surv):
    
    num_cats = len(surv)
    
    def fitness_from_constant_fec(b):
        r = get_leading_eigenvalue(Leslie([b]*num_cats, surv))
        return r - r_desired
    b = find_root(fitness_from_constant_fec, 0, 1)
    
    return [b]*num_cats

fec = get_fec_vector(1, surv)

In [21]:
plot_growth_of_P(fec, surv, 0.1)

####  Now I want to get the survival and fecundity vectors for squirrels. 

In [373]:
def pregnancy_simulation(Z, offspring_threshold, n = 10000, verbose = False, life_history = False, all_details = False):
    nut_findings = [Z.get_random_element() for _ in range(n)]
    offspring_days = [0]*n
    nuts_in_storage = 0
    offspring_count = 0
    pregnant = False
    for k in range(n):
        nut_finding = nut_findings[k]
        nuts_in_storage += nut_finding - 1
        if nuts_in_storage < 0:
            break
        if pregnant:
            offspring_count += 1
            offspring_days[k] = 1
            pregnant = False
        if not pregnant and nuts_in_storage > offspring_threshold:
            pregnant = True
            nuts_in_storage += -1
    to_be_returned = None
    msg = 'Lived until {} - produced {} offspring'.format(k+1, offspring_count)
    bare_facts = (k+1, offspring_count)
    life_hist = offspring_days
    if verbose:
        return(msg)
    if life_history:
        return(life_hist)
    if all_details:
        return([bare_facts, life_hist])
    else:
        return(bare_facts)

In [459]:
def truncate_at_zeroes(some_list, num_zeroes = 5):
    support = [i for i, e in enumerate(some_list) if e != 0]
    truncation_point = max(support) + num_zeroes + 1
    truncated_list = deepcopy(some_list[:truncation_point])
    return(truncated_list)

def birth_death_days(Z, offspring_threshold, num_squirrels = 1000, n = 10000):
    death_days = [0]*n
    birth_days = [0]*n
    for s in range(num_squirrels):
        info = pregnancy_simulation(Z, offspring_threshold, all_details = True)
        dday = info[0][0]
        death_days[dday] += 1
        life_history = info[1]
        for a, b in enumerate(life_history):
            if b == 1:
                birth_days[a] += 1
    num_alive = [num_squirrels]+ [0]*(n-1)
    for a, d in enumerate(death_days):
        if a != 0:
            num_alive[a] = num_alive[a-1] - d
    num_alive = truncate_at_zeroes(num_alive, 1)
    birth_days = birth_days[:len(num_alive)]
    death_days = death_days[:len(num_alive)]
    return(birth_days, death_days, num_alive)
    
    
def squirrel_surv(num_alive):
    s_vec = []
    for a, n in enumerate(num_alive):
        try:
            s = num_alive[a+1]/num_alive[a].n()
            s_vec = s_vec + [s]
        except:
            pass
    return(s_vec)

def squirrel_fec(num_alive, birth_days):
    s_fec = [0]*len(num_alive)
    for a, n in enumerate(num_alive[:-1]):
        s_fec[a] = birth_days[a]/n
    return(s_fec)

def squirrel_fec_surv(Z, offspring_threshold, num_squirrels = 10000, n = 10000):
    b, d, n = birth_death_days(Z, offspring_threshold, num_squirrels)
    s_fec = squirrel_fec(n, b)
    s_surv = squirrel_surv(n)
    return(s_fec, s_surv)

In [545]:
#P = [1, 1, 1.06]  #Gives betahat = 10 as optimal
P = [1, 2, 1.2]
X = GeneralDiscreteDistribution(P)

In [554]:
X.get_random_element()

2

In [546]:
f, s = squirrel_fec_surv(X, 1, 10000)

In [547]:
len(f)

114

In [548]:
L = Leslie(f, s)

In [549]:
get_leading_eigenvalue(L)

1.02883525285627

In [553]:
plot_growth_of_P(f, s, 0.1, verbose = True)

appreciated to 1
appreciated to 2
appreciated to 3
appreciated to 4
appreciated to 5
appreciated to 6
appreciated to 7
appreciated to 8
appreciated to 9
appreciated to 10
appreciated to 11
appreciated to 12
appreciated to 13
appreciated to 14
appreciated to 15
appreciated to 16
appreciated to 17
appreciated to 18
appreciated to 19
appreciated to 20
appreciated to 21
appreciated to 22
appreciated to 23
appreciated to 24
appreciated to 25
appreciated to 26
appreciated to 27
appreciated to 28
appreciated to 29
appreciated to 30
appreciated to 31
appreciated to 32
appreciated to 33
appreciated to 34
appreciated to 35
appreciated to 36
appreciated to 37
appreciated to 38
appreciated to 39
appreciated to 40
appreciated to 41
appreciated to 42
appreciated to 43
appreciated to 44
appreciated to 45
appreciated to 46
appreciated to 47
appreciated to 48
appreciated to 49
appreciated to 50
appreciated to 51
appreciated to 52
appreciated to 53
appreciated to 54
appreciated to 55
appreciated to 56
a

In [513]:
plot_growth_of_P(f[:20], s[:20], 0.1, verbose = True)

appreciated to 1
appreciated to 2
appreciated to 3
appreciated to 4
appreciated to 5
appreciated to 6
appreciated to 7
appreciated to 8
appreciated to 9
appreciated to 10
appreciated to 11
appreciated to 12
appreciated to 13
appreciated to 14
appreciated to 15
appreciated to 16
appreciated to 17
appreciated to 18
appreciated to 19


In [262]:
def bisection(f,a,b,prec = 10^(-13)):
    '''Approximate solution of f(x)=0 on interval [a,b] by bisection method.

    Parameters
    ----------
    f : function
        The function for which we are trying to approximate a solution f(x)=0.
    a,b : numbers
        The interval in which to search for a solution. The function returns
        None if f(a)*f(b) >= 0 since a solution is not guaranteed.
    N : (positive) integer
        The number of iterations to implement.

    Returns
    -------
    x_N : number
        The midpoint of the Nth interval computed by the bisection method. The
        initial interval [a_0,b_0] is given by [a,b]. If f(m_n) == 0 for some
        midpoint m_n = (a_n + b_n)/2, then the function returns this solution.
        If all signs of values f(a_n), f(b_n) and f(m_n) are the same at any
        iteration, the bisection method fails and return None.

    Examples
    --------
    >>> f = lambda x: x**2 - x - 1
    >>> bisection(f,1,2,25)
    1.618033990263939
    >>> f = lambda x: (2*x - 1)*(x - 3)
    >>> bisection(f,0,1,10)
    0.5
    '''
    if f(a)*f(b) >= 0:
        print("Bisection method fails.")
        return None
    a_n = a
    b_n = b
    m_n = 0
    while abs(f(m_n)) > prec:
        m_n = (a_n + b_n)/2
        f_m_n = f(m_n)
        if f(a_n)*f_m_n < 0:
            a_n = a_n
            b_n = m_n
        elif f(b_n)*f_m_n < 0:
            a_n = m_n
            b_n = b_n
        elif f_m_n == 0:
            return m_n
        else:
            print("Bisection method fails.")
            return None
    return(m_n)